# Evaluation of TAE Experiments

This notebook connects to MLflow, downloads all experiment runs and creates visualizations.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
from pprint import pprint

import dagshub
import matplotlib.pyplot as plt
import pandas as pd
from mlflow.client import MlflowClient

from model.common import MRL_DRIVER_NAMES_MAPPING
from model.fonts import set_cmu_typewriter_font
from model.latex import get_caption, pivot_table_to_latex, pivotize_drivers
from model.mlflow import download_all_runs, download_predictions, load_predictions
from model.plot import plot_results

In [ ]:
REPO_NAME = 'driver-tae'
USER_NAME = 'matejfric'
dagshub.init(REPO_NAME, USER_NAME, mlflow=True)  # type: ignore

pd.set_option('display.max_columns', None)

client = MlflowClient()

plt.style.use('seaborn-v0_8-whitegrid')
font = set_cmu_typewriter_font()

## Get all experiment runs

In [ ]:
experiments = client.search_experiments(filter_string="name!='Default'")
pprint([experiment.name for experiment in experiments])

In [ ]:
runs_df = download_all_runs(client=client, experiments=experiments)

# Display the DataFrame
print(f'Total runs: {len(runs_df)}')
runs_df.head()

In [ ]:
integer_columns = ['image_size', 'latent_dim', 'batch_size', 'early_stopping']
runs_df[integer_columns] = runs_df[integer_columns].astype(int)

## Helper functions

In [ ]:
source_type_map = {
    'depth': 'MDE',
    'source_depth': 'Depth',
    'images': 'RGB',
    'rgb': 'RGB',
    'masks': 'Mask',
    'rgbd': 'RGBD',
    'rgbdm': 'RGBDM',
}
source_type_color_map = {
    'Depth': 'tab:red',
    'RGB': 'tab:green',
    'Mask': 'tab:blue',
    'MDE': 'tab:orange',
    'RGBD': 'tab:purple',
    'RGBDM': 'tab:cyan',
}
source_type_linestyle_map = {
    'MDE': '-',
    'Mask': '--',
    'Depth': '-.',
    'RGB': '-',
    'RGBD': '--',
    'RGBDM': '-.',
}
driver_name_mapping = {
    **MRL_DRIVER_NAMES_MAPPING,
    'all': 'All',
    'mean': 'Mean',
    **{str(i): str(i) for i in range(1, 6)},
    **{i: str(i) for i in range(1, 6)},
}

In [ ]:
OUTPUT_DIR = Path('outputs')
OUTPUT_DIR.mkdir(exist_ok=True)
(OUTPUT_DIR / 'dmd_tae').mkdir(exist_ok=True)
(OUTPUT_DIR / 'mrl_tae').mkdir(exist_ok=True)
(OUTPUT_DIR / 'mrl_tae_eff').mkdir(exist_ok=True)

## DMD Filtering

In [ ]:
df_selection = runs_df[
    (runs_df['batch_size'] == 256)
    & (runs_df['dataset'] == 'dmd')
    # source_type 'source_depth' is invalid and replaced with 'source_depth_ir_masked'
    & (runs_df['source_type'] != 'source_depth')
    # Experiment with TimeDistributedV2
    & (runs_df['run_id'] != 'f6aef4dd035340648ec48e696ca46c33')
]
df_selection.loc[
    df_selection['source_type'] == 'source_depth_ir_masked', 'source_type'
] = 'source_depth'

In [ ]:
idx = df_selection.groupby(
    ['driver', 'source_type', 'latent_dim', 'image_size', 'encoder_name']
)['metric.roc_auc'].idxmax()
best_runs_df = df_selection.loc[idx]

df_dmd = best_runs_df[
    (best_runs_df['image_size'] == 64) & (best_runs_df['latent_dim'] == 128)
]
print(df_dmd.shape)
df_dmd[
    [
        'driver',
        'source_type',
        'metric.roc_auc',
        'early_stopping',
        'patience',
        'min_epochs',
        'best_metric',
        'latent_dim',
        'image_size',
        'encoder_name',
    ]
]

In [ ]:
df_dmd_pivot = pivotize_drivers(
    df_dmd,
    source_type_map=source_type_map,
    driver_name_mapping=driver_name_mapping,
)
df_dmd_pivot.head()

In [ ]:
pivot_table_to_latex(
    df_dmd_pivot,
    OUTPUT_DIR / 'dmd_tae_pivot.tex',
    'tab:dmd-tae-pivot',
    get_caption('TAE', 'DMD'),
)

## MRL Filtering

In [ ]:
runs_df['encoder_name'] = runs_df['encoder_name'].fillna('ISVC23EncoderV1')
df_selection = runs_df[
    (runs_df['dataset'] != 'dmd')
    & (
        (runs_df['encoder_name'] == 'EfficientNetEncoder')
        | (runs_df['encoder_name'] == 'ISVC23EncoderV1')
    )
    & (runs_df['source_type'] != 'rgbdm')
]

In [ ]:
idx = df_selection.groupby(
    ['driver', 'source_type', 'latent_dim', 'image_size', 'encoder_name']
)['metric.roc_auc'].idxmax()
best_runs_df = df_selection.loc[idx]

In [ ]:
df = best_runs_df[
    (best_runs_df['image_size'] == 64) & (best_runs_df['latent_dim'] == 128)
]
print(df.shape)
df[
    [
        'driver',
        'source_type',
        'metric.roc_auc',
        'early_stopping',
        'patience',
        'min_epochs',
        'best_metric',
        'latent_dim',
        'image_size',
        'encoder_name',
    ]
]

In [ ]:
df_efficientnet = df[df['encoder_name'] == 'EfficientNetEncoder'].copy()
df_efficientnet[
    [
        'driver',
        'source_type',
        'metric.roc_auc',
        'early_stopping',
        'patience',
        'min_epochs',
        'best_metric',
        'latent_dim',
        'image_size',
        'encoder_name',
    ]
]

In [ ]:
df_isvc = df[df['encoder_name'] == 'ISVC23EncoderV1'].copy()
df_isvc[
    [
        'driver',
        'source_type',
        'metric.roc_auc',
        'early_stopping',
        'patience',
        'min_epochs',
        'best_metric',
        'latent_dim',
        'image_size',
        'encoder_name',
    ]
]

In [ ]:
df_efficientnet_pivot = pivotize_drivers(
    df_efficientnet,
    source_type_map=source_type_map,
    driver_name_mapping=driver_name_mapping,
)
df_efficientnet_pivot = df_efficientnet_pivot[
    [
        'Image Type',
        'Driver 1',
        'Driver 2',
        'Driver 4',
        'Driver 5',
        'Driver 6',
        'Mean',
        'All',
    ]
]
df_efficientnet_pivot.head()

In [ ]:
df_isvc_pivot = pivotize_drivers(
    df_isvc,
    source_type_map=source_type_map,
    driver_name_mapping=driver_name_mapping,
)
df_isvc_pivot = df_isvc_pivot[
    [
        'Image Type',
        'Driver 1',
        'Driver 2',
        'Driver 4',
        'Driver 5',
        'Driver 6',
        'Mean',
        'All',
    ]
]
df_isvc_pivot.head()

In [ ]:
pivot_table_to_latex(
    df_efficientnet_pivot,
    OUTPUT_DIR / 'efficientnet_pivot.tex',
    label='tab:efficientnet-mrl-pivot',
    caption=get_caption('TAE-EfficientNet', 'MRL'),
)

In [ ]:
pivot_table_to_latex(
    df_isvc_pivot,
    OUTPUT_DIR / 'tae_pivot.tex',
    label='tab:tae-mrl-pivot',
    caption=get_caption('TAE', 'MRL'),
)

## Visualizations

### MRL TAE-EfficientNet

In [ ]:
df_efficientnet = download_predictions(client, df_efficientnet)
data_eff = load_predictions(df_efficientnet, source_type_map=source_type_map)

In [ ]:
plot_kwargs = dict(
    source_type_color_map=source_type_color_map,
    driver_name_mapping=driver_name_mapping,
    fig_height_multiplier=7,
    fig_width_multiplier=5.5,
    n_rows=2,
    linewidth=2,
    legend_outside=False,
)
plt.rc('font', size=22)

In [ ]:
# Re-order
data_eff = {
    'geordi': data_eff['geordi'],
    'jakub': data_eff['jakub'],
    'michal': data_eff['michal'],
    'poli': data_eff['poli'],
    'dans': data_eff['dans'],
    'all': data_eff['all'],
}

In [ ]:
plot_results(
    'roc', data_eff, save_path=OUTPUT_DIR / 'mrl_tae_eff' / 'roc_auc.pdf', **plot_kwargs
)

In [ ]:
plot_results(
    'pr', data_eff, save_path=OUTPUT_DIR / 'mrl_tae_eff' / 'pr_auc.pdf', **plot_kwargs
)

### MRL TAE

In [ ]:
df_isvc = download_predictions(client, df_isvc)
data_tae = load_predictions(df_isvc, source_type_map=source_type_map)

In [ ]:
plot_kwargs = dict(
    source_type_color_map=source_type_color_map,
    driver_name_mapping=driver_name_mapping,
    fig_height_multiplier=7,
    fig_width_multiplier=5.5,
    n_rows=2,
    linewidth=2,
    legend_outside=False,
)
plt.rc('font', size=22)

In [ ]:
# Re-order
data_tae = {
    'geordi': data_tae['geordi'],
    'jakub': data_tae['jakub'],
    'michal': data_tae['michal'],
    'poli': data_tae['poli'],
    'dans': data_tae['dans'],
    'all': data_tae['all'],
}

In [ ]:
plot_results(
    'roc', data_tae, save_path=OUTPUT_DIR / 'mrl_tae' / 'roc_auc.pdf', **plot_kwargs
)

In [ ]:
plot_results(
    'pr', data_tae, save_path=OUTPUT_DIR / 'mrl_tae' / 'pr_auc.pdf', **plot_kwargs
)

### DMD TAE

In [ ]:
df_dmd = download_predictions(client, df_dmd)
data_dmd_tae = load_predictions(df_dmd, source_type_map=source_type_map)

In [ ]:
plot_kwargs = dict(
    source_type_color_map=source_type_color_map,
    fig_height_multiplier=7,
    fig_width_multiplier=5.5,
    n_rows=2,
    linewidth=2,
    legend_outside=False,
)
plt.rc('font', size=22)

In [ ]:
plot_results(
    'roc',
    data_dmd_tae,
    save_path=OUTPUT_DIR / 'dmd_tae' / 'roc_auc.pdf',
    **plot_kwargs,
)

In [ ]:
plot_results(
    'pr', data_dmd_tae, save_path=OUTPUT_DIR / 'dmd_tae' / 'pr_auc.pdf', **plot_kwargs
)